In [1]:
import os

In [2]:
%pwd

'e:\\Diabetes_classifier\\research'

In [3]:
os.chdir('../')

In [4]:
from  pathlib import Path
from dataclasses import dataclass


@dataclass
class Model_trainerConfig:
    root_dir :Path
    model_dir :Path
    
    


In [7]:
from src.DIABETES_CLASSIFIER.utils.common import read_yaml,create_directories
from src.DIABETES_CLASSIFIER.constants import config_file_path,param_file_path
from src.DIABETES_CLASSIFIER import logger

In [11]:
class Configuratoration:
    def __init__(self,config=config_file_path,params=param_file_path):
        self.config=read_yaml(config)
        self.params=read_yaml(params)

        create_directories([self.config.artifacts_root])

    def get_model_trainer(self)->Model_trainerConfig:
        
        data=self.config.data_ingestion
        trainer=self.config.data_trainer
        transformation=self.config.data_transformation


        create_directories([trainer.root_dir])

        trainer_model=Model_trainerConfig(
            root_dir=trainer.root_dir,
            model_dir=trainer.model_dir
        )



        return (data,transformation,trainer_model)

In [12]:
from src.DIABETES_CLASSIFIER.components.data_transformation import Data_Transformation
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from catboost import CatBoostClassifier
from xgboost import XGBRFClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,recall_score
from sklearn.neighbors import KNeighborsClassifier
from src.DIABETES_CLASSIFIER.utils.common import  model_predictions,save_model


In [15]:
class Model_trainer:
    def __init__(self,config,data,transformation):
        self.config=config
        self.data=data
        self.transformation=transformation
 

    def get_trained_model(self,x_train,x_test,y_train,y_test):
        models={
            'logistic_model':LogisticRegression(),
            'naive_bayes_model':GaussianNB(),
            'svc':SVC(),
            'Decision_tree':DecisionTreeClassifier(),
            'Kn_neighbours':KNeighborsClassifier(),
            'Random_forest':RandomForestClassifier(),
            'Gradient_boosting':GradientBoostingClassifier(),
            'Cat_boosting':CatBoostClassifier(),
            'Xg_boosting':XGBRFClassifier()}
        

        
        model_list,r_score_list,score_list=model_predictions(models=models,x_train=x_train,x_test=x_test,y_train=y_train,y_test=y_test)
        
        model_report={}

        print(model_list)

        for x,y in zip(model_list,r_score_list):
            model_report[x]=y

        print(model_report)

        high_values_models=sorted(model_report.items(),key=lambda x:x[1],reverse=True)

        print(high_values_models)

        best_model=models[high_values_models[0][0]]

        print(best_model)


        save_model(obj=best_model,file_path=self.config.model_dir)


In [16]:
from src.DIABETES_CLASSIFIER.components.data_transformation import Data_Transformation


In [17]:
try:
    config=Configuratoration()
    data,transformation,trainer_model=config.get_model_trainer()

    transformation=Data_Transformation(config=transformation,ingestion=data)

    x_train,x_test,y_train,y_test=transformation.data_tranformation_initiated()

    print(x_train.shape)
    print(x_test.shape)
    print(y_train.shape)
    print(y_test.shape)

    model_trainer=Model_trainer(config=trainer_model,data=data,transformation=transformation)

    model_trainer.get_trained_model(x_train,x_test,y_train,y_test)

except Exception as e:
    raise e


[2023-08-22 15:53:09,182: INFO: common: safe loaded config\config.yaml]
[2023-08-22 15:53:09,208: INFO: common: safe loaded params.yaml]
[2023-08-22 15:53:09,212: INFO: common: The directory created]
[2023-08-22 15:53:09,215: INFO: common: The directory created]


[2023-08-22 15:53:09,497: INFO: data_transformation: creation of preprocessor_obj]
(76916, 11)
(19230, 11)
(76916,)
(19230,)
Learning rate set to 0.065806
0:	learn: 0.5486590	total: 202ms	remaining: 3m 21s
1:	learn: 0.4608544	total: 288ms	remaining: 2m 23s
2:	learn: 0.3873210	total: 380ms	remaining: 2m 6s
3:	learn: 0.3254048	total: 517ms	remaining: 2m 8s
4:	learn: 0.2779569	total: 695ms	remaining: 2m 18s
5:	learn: 0.2472660	total: 867ms	remaining: 2m 23s
6:	learn: 0.2225951	total: 1.03s	remaining: 2m 26s
7:	learn: 0.1874559	total: 1.23s	remaining: 2m 32s
8:	learn: 0.1704461	total: 1.42s	remaining: 2m 35s
9:	learn: 0.1514158	total: 1.63s	remaining: 2m 41s
10:	learn: 0.1419582	total: 1.82s	remaining: 2m 43s
11:	learn: 0.1339836	total: 2.08s	remaining: 2m 51s
12:	learn: 0.1278116	total: 2.24s	remaining: 2m 50s
13:	learn: 0.1203908	total: 2.36s	remaining: 2m 46s
14:	learn: 0.1132094	total: 2.44s	remaining: 2m 40s
15:	learn: 0.1082172	total: 2.54s	remaining: 2m 35s
16:	learn: 0.1047086	tota

In [20]:
from src.DIABETES_CLASSIFIER.utils.common import load_model

In [21]:
model=load_model(Path('artifacts\model_trainer\model_trainer.pkl'))

In [24]:
model.fit(x_train,y_train)

DecisionTreeClassifier()

In [25]:
from sklearn.metrics import accuracy_score

In [28]:
y_predict=model.predict(x_test)
score =accuracy_score(y_test,y_predict)
print(score)

0.9500260010400416
